<a href="https://colab.research.google.com/github/Noxybot/tt_21/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras import *

SEED = 228

tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

GEO_CLUSTERS = 300
COMPANY_UNITS = 10


In [138]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [130]:
def preprocess (dataframe):
  kmeans = KMeans(n_clusters = GEO_CLUSTERS, init ='k-means++')
  kmeans.fit(dataframe[dataframe.columns[0:2]])
  dataframe['geo_cluster_label'] = kmeans.fit_predict(dataframe[dataframe.columns[0:2]])
  dataframe.loc[dataframe["company"].value_counts()[dataframe["company"]].values < 10, "company"] = -1
  dataframe = dataframe.drop('latitude', axis=1)
  dataframe = dataframe.drop('longitude', axis=1)
  first_X = pd.concat([train_data["is_local"], train_data.iloc[:, 3:7]], axis=1).to_numpy()
  second_X = le.fit_transform(train_data["company"].to_numpy())
  third_X = train_data["type"].to_numpy()
  fourth_X = train_data["geo_cluster_label"].to_numpy()

  out_Y = train_data["target"].to_numpy()
  return first_X, second_X, third_X, fourth_X, out_Y



In [139]:
train_data = train_data.drop(train_data.columns[0], axis=1)
first_X_train, second_X_train, third_X_train, fourth_X_train, out_Y_train = preprocess(train_data)
first_X_test, second_X_test, third_X_test, fourth_X_test, out_Y_test = preprocess(test_data)
#print(train_data.company.value_counts())
#train_data
first_X_test

array([[ 0.00000000e+00,  0.00000000e+00,  3.00000000e+00,
        -1.35060089e+05,  8.60133965e+04],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -1.76684506e+03,  1.49856402e+04],
       [ 0.00000000e+00,  0.00000000e+00,  3.00000000e+00,
        -1.77302874e+05,  4.48819580e+04],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.01012487e+05,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         6.46164703e+03,  0.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.13033390e+05,  0.00000000e+00]])

In [140]:
second_X_train.max()

1483

In [141]:
input_first_branch = Input(shape=(5, ), name="first")
first = Dense(32,)(input_first_branch)
first = BatchNormalization()(first)
first = Dense(32,)(first)
first = BatchNormalization()(first)
first = Model(inputs=input_first_branch, outputs=first)

input_second_branch = Input(shape=(1,), name="second")
second = Embedding(second_X_train.max() + 1, output_dim=128)(input_second_branch)
second = Flatten()(second)
second = Dense(32,)(second)
second = Model(inputs=input_second_branch, outputs=second)

input_third_branch = Input(shape=(1,), name="third")
third = Embedding(third_X_train.max() + 1, output_dim=16)(input_third_branch)
third = Flatten()(third)
third = Dense(16,)(third)
third = Model(inputs=input_third_branch, outputs=third)


input_fourth_branch = Input(shape=(1,), name="fourth")
fourth = Embedding(fourth_X_train.max() + 1, output_dim=128)(input_fourth_branch)
fourth = Flatten()(fourth)
fourth = Dense(32,)(fourth)
fourth = Model(inputs=input_fourth_branch, outputs=fourth)

concat = Concatenate()([first.output, second.output, third.output, fourth.output])

concat = Dense(64,)(concat)
concat = BatchNormalization()(concat)
concat = Dense(64,)(concat)
concat = BatchNormalization()(concat)
output = Dense(1, name="output")(concat)

my_model = Model(inputs=[first.input, second.input, third.input, fourth.input], outputs=output)
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(epochs=10, batch_size=128, x={"first": first_X_train, "second": second_X_train, "third": third_X_train, "fourth": fourth_X_train }, y=out_Y_train)


Epoch 1/10
1251/1251 [==============================] - 12s 8ms/step - loss: 1.4043 - accuracy: 0.7182
Epoch 2/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.8029 - accuracy: 0.7139
Epoch 3/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.6085 - accuracy: 0.7230
Epoch 4/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.5391 - accuracy: 0.7358
Epoch 5/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.5658 - accuracy: 0.7231
Epoch 6/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.8024 - accuracy: 0.7122
Epoch 7/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.7514 - accuracy: 0.7236
Epoch 8/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.5619 - accuracy: 0.7289
Epoch 9/10
1251/1251 [==============================] - 11s 9ms/step - loss: 0.5552 - accuracy: 0.7146
Epoch 10/10
1251/1251 [==============================] - 11s 9ms/step - l

In [142]:
my_model.evaluate(x={"first": first_X_test, "second": second_X_test, "third": third_X_test, "fourth": fourth_X_test }, y=out_Y_test)


5001/5001 [==============================] - 10s 2ms/step - loss: 0.5320 - accuracy: 0.7378


[0.5319505333900452, 0.7377891540527344]